In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

row_data=pd.read_csv('/kaggle/input/titanic/train.csv', index_col='PassengerId')
row_data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [3]:
row_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [4]:
# Filter data using EDA techniques 
row_data=row_data.drop("Cabin",axis=1)

row_data.Ticket=row_data.Ticket.apply(lambda x: x.split(" ")[-1] if len(x.split(' '))>=2 else x)
row_data.Ticket=row_data.Ticket.apply(lambda x:x if x.isdigit() else np.NAN)

row_data=row_data.drop("Name",axis=1)
row_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,21171,7.2500,S
2,1,1,female,38.0,1,0,17599,71.2833,C
3,1,3,female,26.0,0,0,3101282,7.9250,S
4,1,1,female,35.0,1,0,113803,53.1000,S
5,0,3,male,35.0,0,0,373450,8.0500,S


In [5]:
# Filling Nan data
from sklearn.impute import SimpleImputer

numerical_imputer=SimpleImputer(strategy='median')

categorical_imputer=SimpleImputer(strategy='most_frequent')

row_data[['Age','Ticket','Parch']]=numerical_imputer.fit_transform(row_data[['Age','Ticket','Parch']])
row_data[['Sex','Fare','Embarked']]=categorical_imputer.fit_transform(row_data[['Sex','Fare','Embarked']])

row_data.duplicated().sum()

16

In [6]:
row_data.drop_duplicates(inplace=True)
row_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875 entries, 1 to 891
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  875 non-null    int64  
 1   Pclass    875 non-null    int64  
 2   Sex       875 non-null    object 
 3   Age       875 non-null    float64
 4   SibSp     875 non-null    int64  
 5   Parch     875 non-null    float64
 6   Ticket    875 non-null    float64
 7   Fare      875 non-null    object 
 8   Embarked  875 non-null    object 
dtypes: float64(3), int64(3), object(3)
memory usage: 68.4+ KB


In [7]:
# Creating the new col using the parch and sibs
data=row_data
data['Single']=row_data['SibSp']+row_data['Parch']

data['Single']=data['Single'].apply(lambda x:0 if x>0 else 1)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Single
PassengerId,,,,,,,,,,
1,0,3,male,22.0,1,0.0,21171.0,7.25,S,0
2,1,1,female,38.0,1,0.0,17599.0,71.2833,C,0
3,1,3,female,26.0,0,0.0,3101282.0,7.925,S,1
4,1,1,female,35.0,1,0.0,113803.0,53.1,S,0
5,0,3,male,35.0,0,0.0,373450.0,8.05,S,1
...,...,...,...,...,...,...,...,...,...,...
887,0,2,male,27.0,0,0.0,211536.0,13.0,S,1
888,1,1,female,19.0,0,0.0,112053.0,30.0,S,1
889,0,3,female,28.0,1,2.0,6607.0,23.45,S,0


In [8]:
x=data.drop('Survived',axis=1)
y=data.Survived

In [9]:
# applying Scalling on data
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
columns=['Age','Fare','SibSp','Ticket']
x[columns]=scaler.fit_transform(x[columns])
x

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Single
PassengerId,,,,,,,,,
1,3,male,-0.566559,0.554101,0.0,-0.424810,-0.495460,S,0
2,1,female,0.655523,0.554101,0.0,-0.430215,0.786013,C,0
3,3,female,-0.261038,-0.502192,0.0,4.235956,-0.481951,S,1
4,1,female,0.426382,0.554101,0.0,-0.284641,0.422118,S,0
5,3,male,0.426382,-0.502192,0.0,0.108252,-0.479450,S,1
...,...,...,...,...,...,...,...,...,...
887,2,male,-0.184658,-0.502192,0.0,-0.136753,-0.380388,S,1
888,1,female,-0.795699,-0.502192,0.0,-0.287289,-0.040173,S,1
889,3,female,-0.108278,0.554101,2.0,-0.446848,-0.171256,S,0


In [10]:
# using dummies to convert object to python number.
x=pd.get_dummies(x)
x

,Pclass,Age,SibSp,Parch,Ticket,Fare,Single,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,
1,3,-0.566559,0.554101,0.0,-0.424810,-0.495460,0,0,1,0,0,1
2,1,0.655523,0.554101,0.0,-0.430215,0.786013,0,1,0,1,0,0
3,3,-0.261038,-0.502192,0.0,4.235956,-0.481951,1,1,0,0,0,1
4,1,0.426382,0.554101,0.0,-0.284641,0.422118,0,1,0,0,0,1
5,3,0.426382,-0.502192,0.0,0.108252,-0.479450,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,-0.184658,-0.502192,0.0,-0.136753,-0.380388,1,0,1,0,0,1
888,1,-0.795699,-0.502192,0.0,-0.287289,-0.040173,1,1,0,0,0,1
889,3,-0.108278,0.554101,2.0,-0.446848,-0.171256,0,1,0,0,0,1


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier()

model=RFC.fit(x_train,y_train)
RFC.score(x_test,y_test)

0.8171428571428572

In [12]:

testing_data=pd.read_csv("/kaggle/input/titanic/test.csv",index_col='PassengerId')
testing_data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [13]:
testing_data=testing_data.drop("Cabin",axis=1)

testing_data.Ticket=testing_data.Ticket.apply(lambda x: x.split(" ")[-1] if len(x.split(' '))>=2 else x)
testing_data.Ticket=testing_data.Ticket.apply(lambda x:x if x.isdigit() else np.NAN)
testing_data=testing_data.drop("Name",axis=1)

from sklearn.impute import SimpleImputer

numerical_imputer=SimpleImputer(strategy='median')

categorical_imputer=SimpleImputer(strategy='most_frequent')

testing_data[['Age','Ticket','Parch']]=numerical_imputer.fit_transform(testing_data[['Age','Ticket','Parch']])
testing_data[['Sex','Fare','Embarked']]=categorical_imputer.fit_transform(testing_data[['Sex','Fare','Embarked']])
testing_data.info()

tdata=testing_data
tdata['Single']=testing_data['SibSp']+testing_data['Parch']

tdata['Single']=tdata['Single'].apply(lambda x:0 if x>0 else 1)
tdata

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    float64
 5   Ticket    418 non-null    float64
 6   Fare      418 non-null    object 
 7   Embarked  418 non-null    object 
dtypes: float64(3), int64(2), object(3)
memory usage: 29.4+ KB


,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Single
PassengerId,,,,,,,,,
892,3,male,34.5,0,0.0,330911.0,7.8292,Q,1
893,3,female,47.0,1,0.0,363272.0,7.0,S,0
894,2,male,62.0,0,0.0,240276.0,9.6875,Q,1
895,3,male,27.0,0,0.0,315154.0,8.6625,S,1
896,3,female,22.0,1,1.0,3101298.0,12.2875,S,0
...,...,...,...,...,...,...,...,...,...
1305,3,male,27.0,0,0.0,3236.0,8.05,S,1
1306,1,female,39.0,0,0.0,17758.0,108.9,C,1
1307,3,male,38.5,0,0.0,3101262.0,7.25,S,1


In [14]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
columns=['Age','Fare','SibSp','Ticket']
tdata[columns]=scaler.fit_transform(tdata[columns])

tdata=pd.get_dummies(tdata)
tdata

,Pclass,Age,SibSp,Parch,Ticket,Fare,Single,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,
892,3,0.386231,-0.499470,0.0,0.132042,-0.497063,1,0,1,0,1,0
893,3,1.371370,0.616992,0.0,0.187173,-0.511926,0,1,0,0,0,1
894,2,2.553537,-0.499470,0.0,-0.022366,-0.463754,1,0,1,0,1,0
895,3,-0.204852,-0.499470,0.0,0.105198,-0.482127,1,0,1,0,0,1
896,3,-0.598908,0.616992,1.0,4.851741,-0.417151,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,-0.204852,-0.499470,0.0,-0.426193,-0.493105,1,0,1,0,0,1
1306,1,0.740881,-0.499470,0.0,-0.401453,1.314557,1,1,0,1,0,0
1307,3,0.701476,-0.499470,0.0,4.851679,-0.507445,1,0,1,0,0,1


In [15]:
prediction_result=RFC.predict(tdata)
tdata['Survived']=prediction_result
tdata

,Pclass,Age,SibSp,Parch,Ticket,Fare,Single,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived
PassengerId,,,,,,,,,,,,,
892,3,0.386231,-0.499470,0.0,0.132042,-0.497063,1,0,1,0,1,0,0
893,3,1.371370,0.616992,0.0,0.187173,-0.511926,0,1,0,0,0,1,0
894,2,2.553537,-0.499470,0.0,-0.022366,-0.463754,1,0,1,0,1,0,0
895,3,-0.204852,-0.499470,0.0,0.105198,-0.482127,1,0,1,0,0,1,0
896,3,-0.598908,0.616992,1.0,4.851741,-0.417151,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,-0.204852,-0.499470,0.0,-0.426193,-0.493105,1,0,1,0,0,1,0
1306,1,0.740881,-0.499470,0.0,-0.401453,1.314557,1,1,0,1,0,0,1
1307,3,0.701476,-0.499470,0.0,4.851679,-0.507445,1,0,1,0,0,1,0


In [16]:
result=tdata['Survived']
result.to_csv("ALL_Data.csv")